In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'Mnist'
file_name = 'MNIST_CLIP_inference'

df = pd.read_csv(f'./{file_name}.csv')

In [2]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,raw_answer,ill_answer,illless_answer,ill_filter_answer,illless_filter_answer
0,Mnist_1,A field of blooming sunflowers swaying in the ...,low quality,1.5,7,digit 7,illusion digit 7,no illusion digit,illusion digit 7,no illusion digit
1,Mnist_2,A peaceful countryside scene with grazing shee...,low quality,1.5,2,digit 2,illusion digit 2,no illusion digit,illusion digit 2,no illusion digit
2,Mnist_3,A tranquil pond with lily pads floating on the...,low quality,1.5,1,digit 1,illusion digit 1,no illusion digit,illusion digit 1,no illusion digit
3,Mnist_4,A sunny vineyard with rows of ripe grapes,low quality,1.5,0,digit 0,illusion digit 0,no illusion digit,illusion digit 0,no illusion digit
4,Mnist_5,A picturesque vineyard at sunset with the sky ...,low quality,1.5,4,digit 4,illusion digit 4,no illusion digit,illusion digit 4,no illusion digit
...,...,...,...,...,...,...,...,...,...,...
1104,Mnist_1105,Misty jungle surrounded by vibrant flowers and...,low quality,1.5,8,digit 8,illusion digit 8,no illusion digit,illusion digit 8,no illusion digit
1105,Mnist_1106,A forest with blooming flowers,low quality,1.5,0,digit 0,illusion digit 0,no illusion digit,illusion digit 0,no illusion digit
1106,Mnist_1107,"Desolate desert landscape, shifting sands illu...",low quality,1.5,5,digit 5,illusion digit 5,no illusion digit,illusion digit 5,no illusion digit
1107,Mnist_1108,A vast desert with a towering canyon in the di...,low quality,1.5,0,digit 0,illusion digit 0,no illusion digit,illusion digit 0,no illusion digit


In [3]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


# **Raw**

In [4]:
y_true = []
y_pred = []

In [5]:
y_true = df['label'].apply(lambda x: str(x)).tolist()
y_pred = df[f'raw_answer'].apply(lambda x: x.strip().lower()).tolist()

In [6]:
y_pred_ = [item.split()[1] for item in y_pred]

In [7]:
accuracy = accuracy_score(y_true, y_pred_)
precision = precision_score(y_true, y_pred_, average='macro')
recall = recall_score(y_true, y_pred_, average='macro')
f1 = f1_score(y_true, y_pred_, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.9225
Precision: 0.9324
Recall: 0.9222
F1 score: 0.9224


# **Illusion**

In [8]:
y_true = []
y_pred = []

In [9]:
y_pred_illless = [df.at[row, 'illless_answer'].split()[-1] for row in illusionless_indexes]
y_true_illless = ['digit' for i in range(len(y_pred_illless))]

In [10]:
y_true_ill = df['label'].apply(lambda x: str(x)).tolist()
y_pred_ill = df[f'ill_answer'].apply(lambda x: x.strip().lower().split()[-1]).tolist()

In [11]:
y_true = y_true_ill + y_true_illless
y_pred = y_pred_ill + y_pred_illless

In [12]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.918
Precision: 0.9247
Recall: 0.9172
F1 score: 0.9169


# **Filtered**

In [13]:
y_true = []
y_pred = []

In [14]:
y_pred_illless = [df.at[row, 'illless_filter_answer'].split()[-1] for row in illusionless_indexes]
y_true_illless = ['digit' for i in range(len(y_pred_illless))]

In [15]:
y_true_ill = df['label'].apply(lambda x: str(x)).tolist()
y_pred_ill = df[f'ill_filter_answer'].apply(lambda x: x.strip().lower().split()[-1]).tolist()

In [16]:
y_true = y_true_ill + y_true_illless
y_pred = y_pred_ill + y_pred_illless

In [17]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.9155
Precision: 0.9232
Recall: 0.9147
F1 score: 0.9147
